In [1]:
import pandas as pd
from tqdm import tqdm
import json

DEMOGRAPHICS_TO_KEEP = ["SEQN", "SDDSRVYR", "RIAGENDR", "RIDAGEYR", "RIDAGEMN", "RIDAGEEX", "RIDRETH1", "DMDBORN2", "DMDMARTL", "DMDHHSIZ", "DMDFMSIZ", "INDHHIN2", "INDFMIN2", ]

with open("splitting/split_examination.json") as json_file:
    splitting_examination = json.load(json_file)

files_examination = pd.read_feather("../extraction/raw_data/files_examination.feather")

# Drop P_BMX and P_OHXDEN and P_OHXREF: files not accessible and covered by order files name
files_examination.drop(index=files_examination.index[files_examination["data_file_name"].isna() | files_examination["data_file_name"].isin(["P_BMX", "P_OHXDEN", "P_OHXREF"])], inplace=True)

files_examination.set_index("data_file_description", inplace=True)

In [2]:
for category, data_file_description in splitting_examination.items():
    if category in ["body", "eyes", "ears"]:
        continue
    print(category)
    file_names = files_examination.loc[data_file_description, "data_file_name"].drop_duplicates()
    min_seqn = float("inf")
    max_seqn = -float("inf")

    for file_name in tqdm(file_names):
        raw_data = pd.read_csv("../extraction/raw_data/examination/" + file_name + ".csv")
        
        if "SEQN" not in raw_data.columns:
            continue

        if raw_data["SEQN"].min() < min_seqn:
            min_seqn = raw_data["SEQN"].min()
        if max_seqn < raw_data["SEQN"].max():
            max_seqn = raw_data["SEQN"].max()
    
    data_category = pd.DataFrame(None, index=pd.Index(range(int(min_seqn), int(max_seqn) + 1), name="SEQN"))
    
    for file_name in tqdm(file_names):
        raw_data = pd.read_csv("../extraction/raw_data/examination/" + file_name + ".csv")

        if "SEQN" not in raw_data.columns:
            continue

        raw_data.set_index("SEQN", inplace=True)
        raw_data.drop(columns=["file_name", "cycle", "begin_year", "end_year"], inplace=True)

        # object_columns = raw_data.columns[raw_data.dtypes == "object"]
        # raw_data[object_columns] = raw_data[object_columns].astype(str, copy=False)
    
        data_category.loc[raw_data.index, raw_data.columns] = raw_data
    

    # data_category.dropna(how="all").reset_index().to_feather(f"fusionned_data/examination/{category}.feather")
    break

  9%|▉         | 3/32 [00:00<00:01, 24.61it/s]

nose_mouth


 38%|███▊      | 12/32 [00:02<00:05,  3.84it/s]/workspaces/NHANES_preprocessing/env_container/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (92,94,97,102,104,108,109,110,111,113,114,115,116) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
 44%|████▍     | 14/32 [00:03<00:06,  2.73it/s]/workspaces/NHANES_preprocessing/env_container/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (68,72,73,77,78,82,83,84,85,86,87,92,95,97,100,102,103,104,105,108,109,110,111,112,113,115,117) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
100%|██████████| 32/32 [00:11<00:00,  2.68it/s]


In [9]:
data_category.dropna(how="all")

In [ ]:
print(1)

Error: Kernel is dead